In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

device = "cuda:0"
model_path = "DAMO-NLP-SG/VideoLLaMA3-2B"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    device_map={"": device},
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)



2025-02-19 13:04:18.073408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739970258.081883 1579852 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739970258.085667 1579852 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": "/home/ubuntu/temp/mp4s/0.mp4", "fps": 1, "max_frames": 180, "start_time": 0, "end_time": 15}},
            {"type": "text", "text": "Describe the video."},
        ]
    },
    {"role": "assistant", "content": "It is a police academy training video."},
    {"role": "user", "content": "What is the video about?"},
]

inputs = processor(
    conversation=conversation,
    add_system_prompt=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)
output_ids = model.generate(**inputs, max_new_tokens=1024)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)

/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(


A police officer is shown practicing self defense in a room.


In [2]:
from transformers.cache_utils import DynamicCache
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": "/home/ubuntu/temp/mp4s/0.mp4", "fps": 1, "max_frames": 180, "start_time": 0, "end_time": 15}},
            {"type": "text", "text": "Describe the video."},
        ]
    },
    {"role": "assistant", "content": "It is a police academy training video."},
    {"role": "user", "content": "What is the video about?"},
]

inputs = processor(
    conversation=conversation,
    add_system_prompt=True,
    add_generation_prompt=False,
    return_tensors="pt"
)
inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)
kv_cache = DynamicCache()
a = model.forward(**inputs, past_key_values=kv_cache, use_cache=True)
print(kv_cache[0][0].shape)

/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(


torch.Size([1, 2, 4648, 128])


In [4]:
inputs["input_ids"][:,-50:]

tensor([[151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665,
         151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665,
         151665,    198,  74785,    279,   2766,     13, 151645,    198, 151644,
          77091,    198,   2132,    374,    264,   4282,  43345,   4862,   2766,
             13, 151645,    198, 151644,    872,    198,   3838,    374,    279,
           2766,    911,     30, 151645,    198]], device='cuda:0')

In [3]:
tokenizer = processor.tokenizer 
tokens = tokenizer.tokenize("assistant")
input_ids = [tokenizer.convert_tokens_to_ids(tokens)]

token_output_list = []

input_ids = [[77091, 198]]
output = model.forward(input_ids=torch.tensor(input_ids).cuda(), past_key_values=kv_cache, use_cache=True)
print(kv_cache[0][0].shape)

max_idx=torch.argmax(output.logits, dim=-1)
max_idx = max_idx[:,1:]
print(max_idx)
print(tokenizer.convert_ids_to_tokens(max_idx))
token_output_list.append(max_idx[0][0])

torch.Size([1, 2, 4650, 128])
tensor([[32]], device='cuda:0')
['A']


In [4]:
for i in range(10):
    output = model.forward(input_ids=max_idx, past_key_values=kv_cache, use_cache=True)
    max_idx=torch.argmax(output.logits, dim=-1)
    print(tokenizer.convert_ids_to_tokens(max_idx))
    print(max_idx)
    token_output_list.append(max_idx[0][0])

['Ġpolice']
tensor([[4282]], device='cuda:0')
['Ġofficer']
tensor([[9452]], device='cuda:0')
['Ġis']
tensor([[374]], device='cuda:0')
['Ġpracticing']
tensor([[35566]], device='cuda:0')
['Ġself']
tensor([[656]], device='cuda:0')
['Ġdefense']
tensor([[9055]], device='cuda:0')
['Ġwith']
tensor([[448]], device='cuda:0')
['Ġa']
tensor([[264]], device='cuda:0')
['Ġman']
tensor([[883]], device='cuda:0')
['ne']
tensor([[811]], device='cuda:0')


In [5]:
token_output_list

[tensor(32, device='cuda:0'),
 tensor(4282, device='cuda:0'),
 tensor(9452, device='cuda:0'),
 tensor(374, device='cuda:0'),
 tensor(35566, device='cuda:0'),
 tensor(656, device='cuda:0'),
 tensor(9055, device='cuda:0'),
 tensor(448, device='cuda:0'),
 tensor(264, device='cuda:0'),
 tensor(883, device='cuda:0'),
 tensor(811, device='cuda:0')]

In [6]:
tokenizer.decode(token_output_list)

'A police officer is practicing self defense with a manne'

tensor([[4282]], device='cuda:0')


['Ġpolice']

In [11]:
conversation = [
    {"role": "user", "content": "What is the video about?"},
]

inputs = processor(
    conversation=conversation,
    add_system_prompt=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
output_ids = model.generate(input_ids=inputs['input_ids'], max_new_tokens=1024, past_key_values=kv_cache, use_cache=True)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(


The video is about the history of the internet. It starts with a brief overview of how the concept of "the net" was first introduced in 1965, followed by a discussion on how it evolved over time to become the global network that we know today. The video also touches upon the role of key figures and events in shaping the development of the internet, including Tim Berners-Lee's invention of HTTP and HTML, which are still fundamental components of web technology. Additionally, it explores the impact of the internet on society and culture since its inception.


In [10]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
kv_cache[0][0].shape

torch.Size([1, 2, 4640, 128])

In [3]:
from datasets import load_from_disk
dataset = load_from_disk("/home/ubuntu/temp/Sports_dataset")

In [ ]:
from tqdm import tqdm
for sample in tqdm(dataset, total=len(dataset)):
    break
print(idx, len(dataset))
print(sample.keys())

  0%|          | 0/579 [00:00<?, ?it/s]

chunks 579


AttributeError: 'str' object has no attribute 'keys'

In [6]:
data_list = []
for sample in tqdm(dataset, total=len(dataset)):
    video_path = sample["mp4_path"]
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": [
                {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 180, "start_time": 0, "end_time": 15}},
                {"type": "text", "text": "Describe the video."},
            ]
        },
    ]

    inputs = processor(
        conversation=conversation,
        add_system_prompt=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    if "pixel_values" in inputs:
        inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)
    output_ids = model.generate(**inputs, max_new_tokens=1024)
    response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

    data_list.append({"mp4_path": video_path, "description": response})
    print(response)

  0%|          | 0/579 [00:00<?, ?it/s]/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(
  0%|          | 1/579 [00:03<33:43,  3.50s/it]

The video shows various scenes of police officers performing their duties. In the first scene, a man is shown standing in front of a building and another officer stands behind him wearing a mask. The second scene depicts two people walking down a path with bushes on either side. The third scene captures three people sitting at a table working together. The fourth scene features a group of people standing outside. The fifth scene displays a group of uniformed individuals lined up to stand at attention. The sixth scene portrays an officer running after another person. The seventh scene shows a man kneeling over a dummy while a police officer watches from inside a tent.


/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(
  0%|          | 2/579 [00:05<23:08,  2.41s/it]

The video starts with a man driving in his car and then transitions to him talking on the phone. He is wearing glasses and has tattoos on his hands. The background music playing while he talks suggests that it might be an important conversation or discussion.


/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(
  1%|          | 3/579 [00:06<20:41,  2.15s/it]

...

The video shows a group of cars lined up on a race track, waiting for the starting signal. They are all in their respective positions, ready to begin the race once the lights turn green. As soon as the light turns green, they quickly accelerate and start racing around the track at high speed. The drivers skillfully navigate through sharp turns and overtake each other with impressive precision. The sound of engines revving and tires screeching is audible throughout the video, creating an exciting atmosphere. Overall, it's an exhilarating display of automotive prowess and racing skills.


/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(
100%|█████████▉| 578/579 [16:45<00:02,  2.03s/it]

The video shows a group of women playing baseball, with some wearing uniforms and others in casual clothing. One woman is seen throwing a ball while another holds up her glove. The video also includes shots of the players on the field, as well as one player practicing alone.


/home/ubuntu/.cache/huggingface/modules/transformers_modules/DAMO-NLP-SG/VideoLLaMA3-2B/562961752c456a26c46194130f9479dbb01ee582/modeling_videollama3.py:384: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Videollama3Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return super().forward(
100%|██████████| 579/579 [16:49<00:00,  1.74s/it]

The video shows a series of pictures and scenes with football players. It starts with a picture of a man wearing an American football uniform, followed by another player in a red jersey standing in front of a car. Then it shows a black screen with white text that reads "BUT...". Next, there are pictures of different NFL players on the field in their respective jerseys. After that, the video cuts to images of people playing games on the computer. The next scene features several pictures of men in sports uniforms against each other. Finally, the video displays various pictures of people dressed up for football games.


In [8]:
len(data_list)

579

In [12]:
#collate to original dataset
collate_list = []

for sample in data_list:
    data_dict = {}
    data_dict["mp4_path"] = sample["mp4_path"]
    data_dict["chunks"] = [{"activity": {"start": 0, "end": 15, "description": sample["description"]}, "interval": (0,15)}]
    collate_list.append(data_dict)

In [15]:
collate_list[2]

{'mp4_path': '/home/ubuntu/temp/mp4s/10.mp4',
 'chunks': [{'activity': {'start': 0,
    'end': 15,
    'description': 'The video shows a group of men playing cricket in an open field. They are divided into two teams, one wearing white and the other green shirts. The players hit the ball with their bats and run on the ground. In the background, there is a large crowd watching the game from the stands.'},
   'interval': (0, 15)}]}

In [16]:
from datasets import Dataset
filtered_dataset = Dataset.from_list(collate_list)
filtered_dataset.save_to_disk(f"/home/ubuntu/temp/sports_description_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/579 [00:00<?, ? examples/s]